In [1]:
%load_ext sql

In [2]:
%sql sqlite:///./test.db

'Connected: @./test.db'

In [3]:
%sql Create Table Phone_Log(Source_Phone_Nbr Bigint,Destination_Phone_Nbr Bigint,Call_Start_DateTime Datetime) ;

 * sqlite:///./test.db
Done.


[]

In [4]:
%%sql
Insert into Phone_Log Values (2345,6789,'2012-07-01 10:00')
,(2345,1234,'2012-07-01 11:00')
,(2345,4567,'2012-07-01 12:00')
,(2345,4567,'2012-07-01 13:00')
,(2345,6789,'2012-07-01 15:00')
,(3311,7890,'2012-07-01 10:00')
,(3311,6543,'2012-07-01 12:00')
,(3311,1234,'2012-07-01 13:00');

 * sqlite:///./test.db
8 rows affected.


[]

In [6]:
%sql select * from phone_log

 * sqlite:///./test.db
Done.


Source_Phone_Nbr,Destination_Phone_Nbr,Call_Start_DateTime
2345,6789,2012-07-01 10:00
2345,1234,2012-07-01 11:00
2345,4567,2012-07-01 12:00
2345,4567,2012-07-01 13:00
2345,6789,2012-07-01 15:00
3311,7890,2012-07-01 10:00
3311,6543,2012-07-01 12:00
3311,1234,2012-07-01 13:00


 SQL to display the Source_Phone_Nbr and a flag where the flag needs to be set to ‘Y’ if first called number and last called number are the same and ‘N’ if first called number and last called number are different

In [9]:
%%sql
select *,
row_number() over(partition by source_phone_nbr order by call_start_datetime asc) first_call_seq,
row_number() over(partition by source_phone_nbr order by call_start_datetime desc) last_call_seq
from phone_log

 * sqlite:///./test.db
Done.


Source_Phone_Nbr,Destination_Phone_Nbr,Call_Start_DateTime,first_call_seq,last_call_seq
2345,6789,2012-07-01 10:00,1,5
2345,1234,2012-07-01 11:00,2,4
2345,4567,2012-07-01 12:00,3,3
2345,4567,2012-07-01 13:00,4,2
2345,6789,2012-07-01 15:00,5,1
3311,7890,2012-07-01 10:00,1,3
3311,6543,2012-07-01 12:00,2,2
3311,1234,2012-07-01 13:00,3,1


In [12]:
%%sql
select source_phone_nbr,
max(case when first_call_Seq =1 then destination_phone_nbr end) first_call, 
max(case when last_call_Seq =1 then destination_phone_nbr end) last_call from (
select *,
row_number() over(partition by source_phone_nbr order by call_start_datetime asc) first_call_seq,
row_number() over(partition by source_phone_nbr order by call_start_datetime desc) last_call_seq
from phone_log)x
group by source_phone_nbr

 * sqlite:///./test.db
Done.


Source_Phone_Nbr,first_call,last_call
2345,6789,6789
3311,7890,1234


In [17]:
%%sql
select source_phone_nbr,case when first_call=last_call then 'Y' else 'N' end flag  from(
select source_phone_nbr,
max(case when first_call_Seq =1 then destination_phone_nbr end) first_call, 
max(case when last_call_Seq =1 then destination_phone_nbr end) last_call from (
select *,
row_number() over(partition by source_phone_nbr order by call_start_datetime asc) first_call_seq,
row_number() over(partition by source_phone_nbr order by call_start_datetime desc) last_call_seq
from phone_log)x
group by source_phone_nbr)T

 * sqlite:///./test.db
Done.


source_phone_nbr,flag
2345,Y
3311,N
